<a href="https://colab.research.google.com/github/VivekKejriwal/TheSchoolOfAI/blob/main/Assignment_4/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),           #28X28X1 >> 28X28X8 
            nn.ReLU(),                               #Relu activation function
            nn.BatchNorm2d(8),                       #Batch-Normalization to amplify the features extracted
            nn.Conv2d(8,16, 3,padding =1),           #28X28X8 >> 28X28X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,2)                        #28X28X16 >> 14X14X16
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),         #14X14X16 >> 14X14X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,16, 3,padding =1),          #14X14X16 >> 14X14X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,16, 3,padding =1),          #14X14X16 >> 14X14X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,16, 3,padding =1),          #14X14X16 >> 14X14X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,16, 3,padding =1),          #14X14X16 >> 14X14X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,2)                        #14X14X16 >> 7X7X16                 
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3),                    #7X7X16 >> 5X5X16
            nn.ReLU(),
            nn.BatchNorm2d(16),                      
            nn.Conv2d(16, 16, 3),                    #5X5X16 >> 3X3X16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 10, 3)                     #3X3X16 >> 1X1X10
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 10)
        return F.log_softmax(x,dim=1)         #Log softmax for the negative-likelihood loss(NLL) of softmax

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()                         #GPU availability check using CUDA
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))                       #Summary of the model having paramter count and Output shape 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4           [-1, 16, 28, 28]           1,168
              ReLU-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,320
              ReLU-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
             ReLU-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
           Conv2d-14           [-1, 16,

In [ ]:


torch.manual_seed(1)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

#Train-loader for the MNIST dataset of handwritten digits 

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

#Test-loader

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = train_loader
    #pbar = train_loader   Uncomment this for having tqdm view of training

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()     # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)                           # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Test set: Average loss: 0.0388, Accuracy: 9877/10000 (98.77%)


Test set: Average loss: 0.0318, Accuracy: 9896/10000 (98.96%)


Test set: Average loss: 0.0285, Accuracy: 9912/10000 (99.12%)


Test set: Average loss: 0.0292, Accuracy: 9898/10000 (98.98%)


Test set: Average loss: 0.0260, Accuracy: 9917/10000 (99.17%)


Test set: Average loss: 0.0242, Accuracy: 9916/10000 (99.16%)


Test set: Average loss: 0.0224, Accuracy: 9922/10000 (99.22%)


Test set: Average loss: 0.0229, Accuracy: 9927/10000 (99.27%)


Test set: Average loss: 0.0260, Accuracy: 9919/10000 (99.19%)


Test set: Average loss: 0.0268, Accuracy: 9915/10000 (99.15%)


Test set: Average loss: 0.0245, Accuracy: 9914/10000 (99.14%)


Test set: Average loss: 0.0243, Accuracy: 9930/10000 (99.30%)


Test set: Average loss: 0.0197, Accuracy: 9933/10000 (99.33%)


Test set: Average loss: 0.0196, Accuracy: 9940/10000 (99.40%)


Test set: Average loss: 0.0184, Accuracy: 9942/10000 (99.42%)


Test set: Average loss: 0.0184, Accurac